In [ ]:
# Import necessary libraries
# These are the standard tools for data analysis in Python

import pandas as pd  # For working with tabular data (like spreadsheets)
import numpy as np  # For numerical operations and arrays
import matplotlib.pyplot as plt  # For creating visualizations
import seaborn as sns  # For beautiful statistical plots
from datetime import datetime, timedelta

# Configure visualization settings
# This makes our plots look nicer and easier to read
sns.set_style('whitegrid')  # Use a clean grid background
plt.rcParams['figure.figsize'] = (12, 6)  # Default figure size
plt.rcParams['font.size'] = 10  # Default font size

# Display all columns in pandas (don't truncate)
pd.set_option('display.max_columns', None)

print("✓ Libraries imported successfully!")

## Step 1: Load Sample Data

Since we don't have real data yet, let's create a sample dataset that mimics
what UCLA YikYak-style posts might look like. This lets us test our pipeline
and visualizations.

**In a real project**, you would load actual data using:
```python
df = pd.read_csv('data/processed/features.csv')
```

In [ ]:
# Create sample data for demonstration
# This simulates what our processed data would look like

np.random.seed(42)  # For reproducibility

# Generate 1000 sample posts
n_posts = 1000

# Sample post texts (realistic campus social media content)
sample_texts = [
    "love the campus during fall",
    "finals week is brutal",
    "parking situation is terrible",
    "best dining hall on campus",
    "anyone else stressed about midterms",
    "go bruins game day",
    "library is packed cant find seat",
    "weather is perfect today",
    "Powell cat spotted again",
    "need study buddy for chem"
]

# Create sample DataFrame
df = pd.DataFrame({
    'post_id': [f'post_{i:04d}' for i in range(n_posts)],
    
    # Timestamps: Random dates over 3 months
    'timestamp': pd.date_range(start='2025-10-01', periods=n_posts, freq='2H') + 
                 pd.to_timedelta(np.random.randint(-30, 30, n_posts), unit='m'),
    
    # Text: Random selection from sample texts
    'text': np.random.choice(sample_texts, n_posts),
    'cleaned_text': np.random.choice(sample_texts, n_posts),
    
    # Upvotes: Log-normal distribution (realistic for social media)
    'upvotes': np.random.lognormal(2, 1, n_posts).astype(int),
    
    # School
    'school': 'UCLA'
})

# Clip upvotes to reasonable range
df['upvotes'] = df['upvotes'].clip(0, 200)

# Add temporal features
df['hour_of_day'] = df['timestamp'].dt.hour
df['day_of_week'] = df['timestamp'].dt.dayofweek
df['week_of_year'] = df['timestamp'].dt.isocalendar().week
df['is_weekend'] = (df['day_of_week'] >= 5).astype(int)

# Add text features
df['char_count'] = df['cleaned_text'].str.len()
df['word_count'] = df['cleaned_text'].str.split().str.len()

# Add simple sentiment (random for demo)
df['sentiment_score'] = np.random.uniform(-0.3, 0.5, n_posts)

# Add engagement label (top 10%)
engagement_threshold = df['upvotes'].quantile(0.90)
df['is_high_engagement'] = (df['upvotes'] >= engagement_threshold).astype(int)

print(f"✓ Created sample dataset with {len(df)} posts")
print(f"\nDataset shape: {df.shape}")
print(f"Date range: {df['timestamp'].min()} to {df['timestamp'].max()}")

## Step 2: Initial Data Inspection

Let's look at the structure and basic statistics of our data.
This helps us understand what we're working with.

In [ ]:
# Display first few rows
# This gives us a concrete sense of what each row represents
print("Sample Posts:")
print("="*80)
df.head(10)

In [ ]:
# Check data types and missing values
# Missing data can cause problems in machine learning!
print("Data Types and Missing Values:")
print("="*80)
df.info()

In [ ]:
# Summary statistics for numerical columns
# This shows distributions: mean, median, min, max, etc.
print("Summary Statistics:")
print("="*80)
df.describe()

## Step 3: Engagement Distribution

Understanding how upvotes are distributed is crucial for our prediction task.
Social media engagement typically follows a power-law distribution:
most posts get few upvotes, but a small number go viral.

In [ ]:
# Plot upvote distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram of upvotes
axes[0].hist(df['upvotes'], bins=50, color='skyblue', edgecolor='black')
axes[0].set_xlabel('Number of Upvotes')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of Upvotes')
axes[0].axvline(df['upvotes'].mean(), color='red', linestyle='--', 
                label=f'Mean: {df["upvotes"].mean():.1f}')
axes[0].axvline(df['upvotes'].median(), color='green', linestyle='--',
                label=f'Median: {df["upvotes"].median():.1f}')
axes[0].legend()

# Box plot by engagement category
df['engagement_category'] = df['is_high_engagement'].map({0: 'Low', 1: 'High'})
sns.boxplot(data=df, x='engagement_category', y='upvotes', ax=axes[1])
axes[1].set_title('Upvotes by Engagement Category')
axes[1].set_xlabel('Engagement Level')
axes[1].set_ylabel('Number of Upvotes')

plt.tight_layout()
plt.show()

# Print statistics
print(f"Upvote Statistics:")
print(f"  Mean: {df['upvotes'].mean():.2f}")
print(f"  Median: {df['upvotes'].median():.2f}")
print(f"  Std Dev: {df['upvotes'].std():.2f}")
print(f"  Min: {df['upvotes'].min()}")
print(f"  Max: {df['upvotes'].max()}")
print(f"\nHigh engagement threshold (90th percentile): {df['upvotes'].quantile(0.90):.1f}")
print(f"High engagement posts: {df['is_high_engagement'].sum()} ({df['is_high_engagement'].mean()*100:.1f}%)")

## Step 4: Temporal Patterns

When do people post? When do posts get more engagement?
Understanding time-based patterns helps us engineer better features.

In [ ]:
# Posts by hour of day
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# 1. Posts by hour
hourly_counts = df['hour_of_day'].value_counts().sort_index()
axes[0, 0].bar(hourly_counts.index, hourly_counts.values, color='coral')
axes[0, 0].set_xlabel('Hour of Day')
axes[0, 0].set_ylabel('Number of Posts')
axes[0, 0].set_title('Posting Activity by Hour')
axes[0, 0].set_xticks(range(0, 24, 2))

# 2. Average upvotes by hour
hourly_engagement = df.groupby('hour_of_day')['upvotes'].mean().sort_index()
axes[0, 1].plot(hourly_engagement.index, hourly_engagement.values, 
                marker='o', color='purple', linewidth=2)
axes[0, 1].set_xlabel('Hour of Day')
axes[0, 1].set_ylabel('Average Upvotes')
axes[0, 1].set_title('Average Engagement by Hour')
axes[0, 1].set_xticks(range(0, 24, 2))
axes[0, 1].grid(True, alpha=0.3)

# 3. Posts by day of week
day_names = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
daily_counts = df['day_of_week'].value_counts().sort_index()
axes[1, 0].bar(range(7), daily_counts.values, 
               color=['lightblue' if i < 5 else 'lightcoral' for i in range(7)])
axes[1, 0].set_xlabel('Day of Week')
axes[1, 0].set_ylabel('Number of Posts')
axes[1, 0].set_title('Posting Activity by Day')
axes[1, 0].set_xticks(range(7))
axes[1, 0].set_xticklabels(day_names)

# 4. Weekend vs Weekday comparison
weekend_data = df.groupby('is_weekend').agg({
    'upvotes': 'mean',
    'post_id': 'count'
}).rename(columns={'post_id': 'count'})
weekend_labels = ['Weekday', 'Weekend']
x = np.arange(len(weekend_labels))
width = 0.35
axes[1, 1].bar(x - width/2, weekend_data['count'], width, label='Post Count', color='skyblue')
ax2 = axes[1, 1].twinx()
ax2.bar(x + width/2, weekend_data['upvotes'], width, label='Avg Upvotes', color='orange')
axes[1, 1].set_xlabel('Day Type')
axes[1, 1].set_ylabel('Post Count', color='skyblue')
ax2.set_ylabel('Average Upvotes', color='orange')
axes[1, 1].set_title('Weekday vs Weekend Activity')
axes[1, 1].set_xticks(x)
axes[1, 1].set_xticklabels(weekend_labels)
axes[1, 1].legend(loc='upper left')
ax2.legend(loc='upper right')

plt.tight_layout()
plt.show()

## Step 5: Text Characteristics

How long are posts? Does length affect engagement?
Let's explore the relationship between text features and upvotes.

In [ ]:
# Text feature analysis
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# 1. Character count distribution
axes[0, 0].hist(df['char_count'], bins=30, color='lightgreen', edgecolor='black')
axes[0, 0].set_xlabel('Character Count')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].set_title('Distribution of Post Length (Characters)')
axes[0, 0].axvline(df['char_count'].mean(), color='red', linestyle='--',
                   label=f'Mean: {df["char_count"].mean():.1f}')
axes[0, 0].legend()

# 2. Word count distribution
axes[0, 1].hist(df['word_count'], bins=20, color='lightyellow', edgecolor='black')
axes[0, 1].set_xlabel('Word Count')
axes[0, 1].set_ylabel('Frequency')
axes[0, 1].set_title('Distribution of Post Length (Words)')
axes[0, 1].axvline(df['word_count'].mean(), color='red', linestyle='--',
                   label=f'Mean: {df["word_count"].mean():.1f}')
axes[0, 1].legend()

# 3. Length vs Engagement scatter
high_eng = df[df['is_high_engagement'] == 1]
low_eng = df[df['is_high_engagement'] == 0]
axes[1, 0].scatter(low_eng['char_count'], low_eng['upvotes'], 
                   alpha=0.3, s=20, label='Low Engagement', color='blue')
axes[1, 0].scatter(high_eng['char_count'], high_eng['upvotes'],
                   alpha=0.5, s=30, label='High Engagement', color='red')
axes[1, 0].set_xlabel('Character Count')
axes[1, 0].set_ylabel('Upvotes')
axes[1, 0].set_title('Post Length vs Engagement')
axes[1, 0].legend()

# 4. Sentiment distribution
axes[1, 1].hist(df['sentiment_score'], bins=30, color='lightpink', edgecolor='black')
axes[1, 1].set_xlabel('Sentiment Score')
axes[1, 1].set_ylabel('Frequency')
axes[1, 1].set_title('Distribution of Sentiment Scores')
axes[1, 1].axvline(0, color='black', linestyle='--', label='Neutral')
axes[1, 1].axvline(df['sentiment_score'].mean(), color='red', linestyle='--',
                   label=f'Mean: {df["sentiment_score"].mean():.3f}')
axes[1, 1].legend()

plt.tight_layout()
plt.show()

# Print correlation
print("Correlations with Upvotes:")
print(f"  Character Count: {df['char_count'].corr(df['upvotes']):.3f}")
print(f"  Word Count: {df['word_count'].corr(df['upvotes']):.3f}")
print(f"  Sentiment Score: {df['sentiment_score'].corr(df['upvotes']):.3f}")

## Step 6: Feature Correlations

Let's create a correlation matrix to see which features are related.
This helps us understand:
- Which features might be redundant (highly correlated)
- Which features might be good predictors (correlated with target)

In [ ]:
# Select numerical features for correlation analysis
feature_cols = ['hour_of_day', 'day_of_week', 'is_weekend', 'char_count', 
                'word_count', 'sentiment_score', 'upvotes', 'is_high_engagement']

correlation_matrix = df[feature_cols].corr()

# Plot correlation heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm',
            center=0, square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Feature Correlation Matrix', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\nKey Insights:")
print("- Values close to +1: Strong positive correlation")
print("- Values close to -1: Strong negative correlation")
print("- Values close to 0: No linear relationship")
print("\nLook for features strongly correlated with 'is_high_engagement'!")

## Step 7: Top Posts Analysis

Let's look at the actual content of high-engagement posts.
This qualitative analysis can reveal patterns that numbers alone won't show.

In [ ]:
# Get top posts by engagement
top_posts = df.nlargest(10, 'upvotes')[['text', 'upvotes', 'sentiment_score', 
                                        'hour_of_day', 'day_of_week']]

print("Top 10 Most Engaged Posts:")
print("="*80)
for idx, (i, row) in enumerate(top_posts.iterrows(), 1):
    day_name = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'][row['day_of_week']]
    print(f"{idx}. [{row['upvotes']} upvotes] {row['text']}")
    print(f"   Posted: {day_name} at {row['hour_of_day']}:00 | "
          f"Sentiment: {row['sentiment_score']:.3f}")
    print()

## Step 8: Summary and Next Steps

Based on this exploratory analysis, we can make informed decisions about:

1. **Feature Selection**: Which features to include in our model
2. **Feature Engineering**: What new features might be useful
3. **Model Choice**: What type of model might work best
4. **Data Quality**: Whether we need more cleaning or filtering

Let's summarize our findings:

In [ ]:
print("EXPLORATORY DATA ANALYSIS SUMMARY")
print("="*80)

print("\n📊 Dataset Overview:")
print(f"  Total posts: {len(df):,}")
print(f"  Date range: {df['timestamp'].min().date()} to {df['timestamp'].max().date()}")
print(f"  Time span: {(df['timestamp'].max() - df['timestamp'].min()).days} days")

print("\n🎯 Engagement Metrics:")
print(f"  High engagement threshold: {df['upvotes'].quantile(0.90):.0f} upvotes")
print(f"  High engagement posts: {df['is_high_engagement'].sum()} ({df['is_high_engagement'].mean()*100:.1f}%)")
print(f"  Average upvotes: {df['upvotes'].mean():.1f}")
print(f"  Median upvotes: {df['upvotes'].median():.1f}")

print("\n⏰ Temporal Patterns:")
peak_hour = df.groupby('hour_of_day')['upvotes'].mean().idxmax()
peak_day = df.groupby('day_of_week')['upvotes'].mean().idxmax()
day_names = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
print(f"  Peak engagement hour: {peak_hour}:00")
print(f"  Peak engagement day: {day_names[peak_day]}")
print(f"  Weekend posts: {df['is_weekend'].sum()} ({df['is_weekend'].mean()*100:.1f}%)")

print("\n📝 Text Characteristics:")
print(f"  Average post length: {df['char_count'].mean():.1f} characters")
print(f"  Average word count: {df['word_count'].mean():.1f} words")
print(f"  Average sentiment: {df['sentiment_score'].mean():.3f}")

print("\n🔍 Key Correlations with Engagement:")
correlations = df[feature_cols].corr()['is_high_engagement'].sort_values(ascending=False)
for feat, corr in correlations.items():
    if feat != 'is_high_engagement':
        print(f"  {feat}: {corr:+.3f}")

print("\n✅ Next Steps:")
print("  1. Train engagement prediction model with these features")
print("  2. Perform topic modeling to identify content themes")
print("  3. Extract post archetypes from high-engagement content")
print("  4. Evaluate model performance and iterate")

print("\n" + "="*80)

## 🎓 Learning Reflection

**What did we learn from this EDA?**

1. **Data Structure**: We understand the schema and what each field represents
2. **Distributions**: We see how engagement follows a power-law (few viral posts)
3. **Temporal Patterns**: Posting behavior varies by hour and day
4. **Text Features**: Post length and sentiment might influence engagement
5. **Correlations**: We identified which features relate to our prediction target

**Why is EDA valuable?**

- Prevents building models on broken data
- Reveals features we might have missed
- Helps set realistic expectations for model performance
- Guides feature engineering decisions
- Often uncovers insights that are valuable on their own!

---

*Remember: Understanding your data is just as important as building fancy models!*